In [1]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler

data = pd.read_csv("../data/datos_grasas_Tec.csv", encoding="latin1")

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 41 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   idDatosGrasas                             51 non-null     int64  
 1   codigoGrasa                               51 non-null     object 
 2   Aceite Base                               47 non-null     object 
 3   Espesante                                 45 non-null     object 
 4   Grado NLGI Consistencia                   51 non-null     float64
 5   Viscosidad del Aceite Base a 40°C. cSt    50 non-null     float64
 6   Penetración de Cono a 25°C, 0.1mm         51 non-null     object 
 7   Punto de Gota, °C                         51 non-null     object 
 8   Estabilidad Mecánica, %                   48 non-null     object 
 9   Punto de Soldadura Cuatro Bolas, kgf      50 non-null     float64
 10  Desgaste Cuatro Bolas, mm               

In [2]:
cols = ["idDatosGrasas",
        "codigoGrasa",
        "Aceite Base",
        "Espesante",
        "Grado NLGI Consistencia",
        "Viscosidad del Aceite Base a 40C. cSt",
        "Penetracion de Cono a 25C, 0.1mm",
        "Punto de Gota, C",
        "Estabilidad Mecanica, %",
        "Punto de Soldadura Cuatro Bolas, kgf",
        "Desgaste Cuatro Bolas, mm",
        "Indice de Carga-Desgaste",
        "Carga Timken Ok, lb",
        "Resistencia al Lavado por Agua a 80C, %",
        "Corrion EMCOR",
        "Corrosion al Cobre",
        "Prueba FZG",
        "Registro NSF",
        "Solidos Totales, %",
        "Tamano de Particula, u",
        "Bombeabilidad",
        "Clasificacion ISO 6743-9",
        "Presion de Flujo a -30C, mbar",
        "Separacion del Aceite, %",
        "Prevencion a la Corrosion",
        "Estabilidad a la Oxidacion",
        "Resistencia a la Humedad",
        "pH",
        "Factor de Torque",
        "Viscosidad Dinamica a 25C, cP",
        "Biodegradabilidad en 28 dias (> 60%)",
        "Factor de Velocidad",
        "Temperatura de Servicio C, min",
        "Temperatura de Servicio C, max",
        "categoria",
        "subtitulo",
        "descripcion",
        "beneficios",
        "aplicaciones",
        "color",
        "textura"
]

data_pdfs = pd.read_csv("../data/datos_grasas_pdfs.csv", encoding="latin1")

data.columns = cols
data_pdfs.columns = cols

data = pd.concat([data, data_pdfs], axis=0)

data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 65 entries, 0 to 13
Data columns (total 41 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   idDatosGrasas                            65 non-null     int64  
 1   codigoGrasa                              65 non-null     object 
 2   Aceite Base                              49 non-null     object 
 3   Espesante                                59 non-null     object 
 4   Grado NLGI Consistencia                  64 non-null     object 
 5   Viscosidad del Aceite Base a 40C. cSt    63 non-null     float64
 6   Penetracion de Cono a 25C, 0.1mm         65 non-null     object 
 7   Punto de Gota, C                         63 non-null     object 
 8   Estabilidad Mecanica, %                  48 non-null     object 
 9   Punto de Soldadura Cuatro Bolas, kgf     63 non-null     float64
 10  Desgaste Cuatro Bolas, mm                39 non-null     

---
# Juan Pablo

In [3]:
data["Grado NLGI Consistencia"] = data["Grado NLGI Consistencia"].replace({
    "M1": 0,
    "M2": 0,
    "M3": 0
}).astype(float)

In [4]:
def arreglar_rangos(data, columna):
    col1 = []
    col2 = []
    
    for i in data[columna]:
        col1.append(i[:3])
        col2.append(i[-3:])
    
    data.insert(loc=len(data.columns), column=columna+" min", value=col1)
    data.insert(loc=len(data.columns), column=columna+" max", value=col2)
    
    data[columna+" min"] = pd.to_numeric(data[columna+" min"])
    data[columna+" max"] = pd.to_numeric(data[columna+" max"])
    
    data.drop(columns=[columna], inplace=True)

arreglar_rangos(data, "Penetracion de Cono a 25C, 0.1mm")

---
# Camila

In [5]:
def arreglar_desigualdades(data, columna, numero_rangos=5):
    filtro = pd.to_numeric(data[columna], errors='coerce').notna()
    
    data_numeros = data[filtro]
    data_objetos = data[~filtro]

    numeros = data_numeros[columna].unique().tolist()
    objetos = data_objetos[columna].unique().tolist()
    
    for i in objetos:
        i_str = str(i)
        if i_str[:1] == "<" or i_str[:1] == ">":
            numeros.append(pd.to_numeric(i_str[1:]))

    rangos = np.linspace(min(pd.to_numeric(numeros)), max(pd.to_numeric(numeros)), numero_rangos - 1)
    rangos = np.linspace(min(rangos) - (rangos[1] - rangos[0]), max(rangos) + (rangos[1] - rangos[0]), numero_rangos + 1)
    
    columnas = {}
    
    for i in range(numero_rangos):
        columnas[f"{rangos[i]} - {rangos[i+1]}"] = []
        
    if "No Gotea" in objetos:
        No_Gotea = []
    
    for i in data[columna].tolist():
        for j in range(numero_rangos):
            i_str = str(i)
            if (not pd.isna(pd.to_numeric(i_str, errors='coerce'))) and (rangos[j] < pd.to_numeric(i_str) <= rangos[j+1]):
                columnas[f"{rangos[j]} - {rangos[j+1]}"].append(1)
            elif (i_str[:1] == "<") and (pd.to_numeric(i_str[1:]) > rangos[j+1]):
                columnas[f"{rangos[j]} - {rangos[j+1]}"].append(1)
            elif (i_str[:1] == ">") and (pd.to_numeric(i_str[1:]) < rangos[j]):
                columnas[f"{rangos[j]} - {rangos[j+1]}"].append(1)
            else:
                columnas[f"{rangos[j]} - {rangos[j+1]}"].append(0)
                
        if ("No Gotea" in objetos) and (i == "No Gotea"):
            No_Gotea.append(1)
        elif ("No Gotea" in objetos) and (i != "No Gotea"):
            No_Gotea.append(0)
    
    for i in range(numero_rangos):
        data.insert(loc=len(data.columns), column=columna+f" {rangos[i]:.2f} - {rangos[i+1]:.2f}", value=columnas[f"{rangos[i]} - {rangos[i+1]}"])
        data[columna+f" {rangos[i]:.2f} - {rangos[i+1]:.2f}"] = pd.to_numeric(data[columna+f" {rangos[i]:.2f} - {rangos[i+1]:.2f}"])
        
    if "No Gotea" in objetos:
        data.insert(loc=len(data.columns), column=columna+" No Gotea", value=No_Gotea)
        data[columna+" No Gotea"] = pd.to_numeric(data[columna+" No Gotea"])
        
    data.drop(columns=[columna], inplace=True)

arreglar_desigualdades(data, "Punto de Gota, C")
arreglar_desigualdades(data, "Estabilidad Mecanica, %")
arreglar_desigualdades(data, "Carga Timken Ok, lb")
arreglar_desigualdades(data, "Resistencia al Lavado por Agua a 80C, %")

---
# Demian

In [6]:
def codificador(data, columna):
    columnas_nuevas = pd.get_dummies(data[columna], prefix=columna)
    columnas_nuevas = columnas_nuevas.astype(int)
    
    data[columnas_nuevas.columns] = columnas_nuevas
    data.drop(columns=[columna], inplace=True)

codificador(data, "Aceite Base")
codificador(data, "Espesante")
codificador(data, "Clasificacion ISO 6743-9")
codificador(data, "color")
codificador(data, "textura")

---
# Alfredo

In [7]:
# eliminar columnas tipo objeto con pocas observaciones
data.drop(columns=["codigoGrasa",
                   "Corrion EMCOR",
                   "Corrosion al Cobre",
                   "Bombeabilidad",
                   "Separacion del Aceite, %",
                   "Prevencion a la Corrosion",
                   "Resistencia a la Humedad",
                   "categoria"], inplace=True)

# eliminar columnas tipo numerico con pocas observaciones
data.drop(columns=["Indice de Carga-Desgaste",
                   "Registro NSF",
                   "Presion de Flujo a -30C, mbar"], inplace=True)

# eliminar columnas 0 non-null
data.drop(columns=["Prueba FZG",
                   "Solidos Totales, %",
                   "Tamano de Particula, u",
                   "Estabilidad a la Oxidacion",
                   "pH",
                   "Factor de Torque",
                   "Viscosidad Dinamica a 25C, cP",
                   "Biodegradabilidad en 28 dias (> 60%)"], inplace=True)

In [4]:
import pandas as pd
data_sucios = pd.read_csv("../data/datos_grasas_Tec.csv", encoding="latin1")

# --- Eliminar columnas tipo objeto con pocas observaciones ---
data_sucios.drop(columns=[
    "codigoGrasa",
    "Corrion EMCOR",
    "Corrosión al Cobre",
    "Bombeabilidad",
    "Separacion del Aceite, %",
    "Prevencion a la Corrosion",
    "Resistencia a la Humedad",
    "categoria"
], inplace=True, errors='ignore')

# --- Eliminar columnas tipo numérico con pocas observaciones ---
data_sucios.drop(columns=[
    "Indice de Carga-Desgaste",
    "Registro NSF",
    "Presion de Flujo a -30°C, mbar"
], inplace=True, errors='ignore')

# --- Eliminar columnas con 0 non-null ---
data_sucios.drop(columns=[
    "Prueba FZG",
    "Solidos Totales, %",
    "Tamaño de Particula, µ",
    "Estabilidad a la Oxidacion",
    "pH",
    "Factor de Torque",
    "Viscosidad Dinámica a 25°C, cP",
    "Biodegradabilidad en 28 dias (> 60%)"
], inplace=True, errors='ignore')

# Guardar el DataFrame limpio en un nuevo CSV
data_sucios.to_csv("datos_grasas_Tec_limpios.csv", index=False)

print("✅ Archivo 'datos_grasas_Tec_limpios.csv' creado correctamente.")


✅ Archivo 'datos_grasas_Tec_limpios.csv' creado correctamente.


In [8]:
for col in data.columns:
    nulos = data[col].notnull().sum()
    filas = len(data)
    dtype = data[col].dtype

    if nulos != filas and dtype != 'object':
        data[col] = data[col].fillna(0)
        print(f"{dtype}, {col}")

float64, Grado NLGI Consistencia
float64, Viscosidad del Aceite Base a 40C. cSt
float64, Punto de Soldadura Cuatro Bolas, kgf
float64, Desgaste Cuatro Bolas, mm
float64, Factor de Velocidad
float64, Temperatura de Servicio C, min
float64, Temperatura de Servicio C, max


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 65 entries, 0 to 13
Data columns (total 70 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   idDatosGrasas                                         65 non-null     int64  
 1   Grado NLGI Consistencia                               65 non-null     float64
 2   Viscosidad del Aceite Base a 40C. cSt                 65 non-null     float64
 3   Punto de Soldadura Cuatro Bolas, kgf                  65 non-null     float64
 4   Desgaste Cuatro Bolas, mm                             65 non-null     float64
 5   Factor de Velocidad                                   65 non-null     float64
 6   Temperatura de Servicio C, min                        65 non-null     float64
 7   Temperatura de Servicio C, max                        65 non-null     float64
 8   subtitulo                                             51 non-null  

---
# Gonzalo

In [10]:
STOPWORDS_ES = [
    "a", "acá", "ahí", "al", "algo", "alguna", "algunas", "alguno", "algunos", 
    "allá", "allí", "ante", "antes", "aquí", "arriba", "así", "aun", "aunque",
    "bajo", "bastante", "bien", "cada", "casi", "como", "con", "cual",
    "cuales", "cuando", "cuanto", "cuantos", "de", "del", "demasiado",
    "demás", "dentro", "desde", "donde", "dos", "el", "él", "ella", "ellas", 
    "ellos", "en", "encima", "entonces", "entre", "era", "eran", "eres", "es",
    "esa", "esas", "ese", "eso", "esos", "esta", "estaba", "estado", "estados",
    "estamos", "están", "este", "estos", "estoy", "fue", "fueron", "fui",
    "fuimos", "ha", "haber", "había", "han", "hasta", "hay", "la", "las", "le",
    "les", "lo", "los", "más", "me", "mi", "mis", "mientras", "muy", "nada",
    "ni", "no", "nos", "nosotros", "nuestra", "nuestros", "o", "os", "otra",
    "otras", "otro", "otros", "para", "pero", "poco", "por", "porque", "que",
    "quien", "quienes", "se", "sea", "ser", "si", "sido", "siempre", "sin",
    "sobre", "solo", "su", "sus", "tal", "también", "tanto", "te", "tener",
    "ti", "tiene", "tienen", "todo", "todos", "tu", "tus", "un", "una", "unas",
    "uno", "unos", "usted", "ustedes", "va", "vamos", "van", "varios", "vaya",
    "veces", "voy", "ya", "yo"
]

def vectorizar(data, columna):
    textos = data[columna].fillna("").astype(str).tolist()

    count_vec = CountVectorizer(stop_words=STOPWORDS_ES)
    count_matrix = count_vec.fit_transform(textos)
    count_df = pd.DataFrame(
        count_matrix.toarray(),
        columns=[f"{columna}_count_{w}" for w in count_vec.get_feature_names_out()],
        index=data.index
    )

    tfidf_vec = TfidfVectorizer(stop_words=STOPWORDS_ES)
    tfidf_matrix = tfidf_vec.fit_transform(textos)
    tfidf_df = pd.DataFrame(
        tfidf_matrix.toarray(),
        columns=[f"{columna}_tfidf_{w}" for w in tfidf_vec.get_feature_names_out()],
        index=data.index
    )

    nuevos = pd.concat([count_df, tfidf_df], axis=1)
    data = data.drop(columns=[columna])
    data = pd.concat([data, nuevos], axis=1)

    return data

data = vectorizar(data, "subtitulo")
data = vectorizar(data, "descripcion")
data = vectorizar(data, "beneficios")
data = vectorizar(data, "aplicaciones")

In [11]:
no_numericas = []

for i in data.columns:
    if data[i].dtype not in ['int64', 'float64', 'int32', 'float32']:
        no_numericas.append(i)
        print()
        print(len(data[i]))
        print(10*"=")
        print(f"{i}: {data[i].dtype}, {data[i].nunique()}")
        print(10*"=")
        for j in data[i].unique():
            print(j)

print()
print(no_numericas)


[]


In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 65 entries, 0 to 13
Columns: 2328 entries, idDatosGrasas to aplicaciones_tfidf_ã³ptima
dtypes: float64(1138), int64(1190)
memory usage: 1.2 MB


---
# Juan Pablo

In [13]:
scaler = MinMaxScaler()

data_scaled = pd.DataFrame(
    scaler.fit_transform(data),
    columns=data.columns,
    index=data.index
)

cols = data.columns[1:]
data[cols] = scaler.fit_transform(data[cols])

grasas_interlub = data[:51]
grasas_pdfs = data[51:]

---
Ya no

In [14]:
from sklearn.metrics.pairwise import cosine_similarity

def similitud_coseno(vector_a, vector_b):
    return cosine_similarity([list(vector_a[1:])], [list(vector_b[1:])])[0][0], vector_b[0]

for idx, row in grasas_pdfs.iterrows():
    similares = []
    
    for indice, fila in grasas_interlub.iterrows():
        similitud, id_parecido = similitud_coseno(list(row), list(fila))
        similares.append((similitud, id_parecido))
        
    similares.sort(reverse=True, key=lambda x: x[0])
    top_ids = [np.int64(x[1]) for x in similares[:5]]
    
    print()
    print("="*20)
    print(f"Grasa {row.iloc[0]}")
    cont = 1
    for i in top_ids:
        print(f"{cont}. {i}")
        cont += 1
    print("="*20)
    print()


Grasa 52.0
1. 15
2. 51
3. 50
4. 33
5. 1


Grasa 53.0
1. 44
2. 45
3. 30
4. 46
5. 31


Grasa 54.0
1. 44
2. 20
3. 45
4. 30
5. 46


Grasa 55.0
1. 44
2. 20
3. 45
4. 46
5. 31


Grasa 56.0
1. 46
2. 45
3. 33
4. 32
5. 31


Grasa 57.0
1. 43
2. 19
3. 4
4. 34
5. 35


Grasa 58.0
1. 43
2. 19
3. 4
4. 50
5. 51


Grasa 59.0
1. 19
2. 43
3. 15
4. 4
5. 20


Grasa 60.0
1. 19
2. 20
3. 43
4. 15
5. 34


Grasa 61.0
1. 19
2. 43
3. 20
4. 15
5. 4


Grasa 62.0
1. 19
2. 43
3. 15
4. 4
5. 51


Grasa 63.0
1. 13
2. 14
3. 40
4. 41
5. 42


Grasa 64.0
1. 14
2. 13
3. 42
4. 41
5. 40


Grasa 65.0
1. 27
2. 28
3. 18
4. 21
5. 39

